# Global Budget Spending Analysis
----
<i> Project 1 Team 5 </i><br/> 
Brett Fuller, Diego Jones, Mav Sanchez

In [1]:
#Import necessary package dependencies
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

from pprint import pprint
from scipy.stats import linregress

#Import API Key
from config import *
from functions import *

#Define directories
source_data_dir = "source_data"
staging_data_dir = "staging_data"
target_data_dir = "target_data"
target_images_dir = "target_images"

### Data extraction
----
#####            API Calls

In [18]:
# Define base URLs for API
base_url_un ="put main endpoint here"
base_url_oecd ="http://stats.oecd.org/restsdmx/sdmx.ashx/GetDataStructure/SNA_TABLE11_SNA93"
base_url_worldbank = "http://api.worldbank.org/v2/"

# Parameters customization

# OECD
#url = "https://stats.oecd.org/SDMX-JSON/data/SNA_TABLE11_SNA93/AUS+AUT+BEL+CAN+CHL+CZE+DNK+EST+FIN+FRA+DEU+GRC+HUN+ISL+IRL+ISR+ITA+JPN+KOR+LUX+NLD+NOR+POL+PRT+SVK+SVN+ESP+SWE+CHE+TUR+GBR+USA+NMEC+RUS.TLYCG+P3CG+D1CG+D3CG+D4CG+D4CGTOS1311+D4CGTOS1312+D4CGTOS1313+D4CGTOS1314+D62_D631XXCG+D62CG+P2_D29D5D8CG+P2CG+D29D5D8CG+D7CG+D7CGTOS1311+D7CGTOS1312+D7CGTOS1313+D7CGTOS1314+D9CG+D9CGTOS1311+D9CGTOS1312+D9CGTOS1313+D9CGTOS1314+D92CG+P5_K2CG+P5CG+P51CG+K2CG.T+010+0101+0102+0103+0104+0105+0106+0107+0108+020+0201+0202+0203+0204+0205+030+0301+0302+0303+0304+0305+0306+040+0401+0402+0403+0404+0405+0406+0407+0408+0409+050+0501+0502+0503+0504+0505+0506+060+0601+0602+0603+0604+0605+0606+070+0701+0702+0703+0704+0705+0706+080+0801+0802+0803+0804+0805+0806+090+0901+0902+0903+0904+0905+0906+0907+0908+100+1001+1002+1003+1004+1005+1006+1007+1008+1009.GS13+GS1311+GS1312+GS1313+GS1314.C+CPC/all?startTime=2009&endTime=2015&dimensionAtObservation=allDimensions"
#url = "https://stats.oecd.org/SDMX-JSON/data/SNA_TABLE11_SNA93/AUS+AUT+BEL+CAN+CHL+CZE+DNK+EST+FIN+FRA+DEU+GRC+HUN+ISL+IRL+ISR+ITA+JPN+KOR+LUX+NLD+NOR+POL+PRT+SVK+SVN+ESP+SWE+CHE+TUR+GBR+USA+NMEC+RUS.TLYCG+P3CG+D1CG+D3CG+D4CG+D4CGTOS1311+D4CGTOS1312+D4CGTOS1313+D4CGTOS1314+D62_D631XXCG+D62CG+P2_D29D5D8CG+P2CG+D29D5D8CG+D7CG+D7CGTOS1311+D7CGTOS1312+D7CGTOS1313+D7CGTOS1314+D9CG+D9CGTOS1311+D9CGTOS1312+D9CGTOS1313+D9CGTOS1314+D92CG+P5_K2CG+P5CG+P51CG+K2CG.T+010+0101+0102+0103+0104+0105+0106+0107+0108+020+0201+0202+0203+0204+0205+030+0301+0302+0303+0304+0305+0306+040+0401+0402+0403+0404+0405+0406+0407+0408+0409+050+0501+0502+0503+0504+0505+0506+060+0601+0602+0603+0604+0605+0606+070+0701+0702+0703+0704+0705+0706+080+0801+0802+0803+0804+0805+0806+090+0901+0902+0903+0904+0905+0906+0907+0908+100+1001+1002+1003+1004+1005+1006+1007+1008+1009.GS13+GS1311+GS1312+GS1313+GS1314.C+CPC/all?startTime=2009&endTime=2015&dimensionAtObservation=allDimensions"

#World Bank
api_call_wb="country/all/indicator/SE.XPD.TOTL.GD.ZS"
api_format_wb = "json"
years_wb = "2006:2019"

In [43]:
# Get the list of Government expenditure on education, total (% of GDP)
response = requests.get(f"{base_url_worldbank}{api_call_wb}?format={api_format_wb}&date={years_wb}").json()
observation_size = response[0]['total']

wb_response = requests.get(f"{base_url_worldbank}{api_call_wb}?format={api_format_wb}&date={years_wb}&per_page={observation_size}").json()
wb_data_df = pd.DataFrame(wb_response[1])

wb_data_df.drop(columns=['indicator', 'country', 'unit','obs_status', 'decimal'], axis=1, inplace=True)
wb_data_df.rename(columns={'countryiso3code':'Country','date':'Year', 'value': 'Percent'}, inplace=True)
wb_data_df.Year = wb_data_df.Year.astype('int64')
wb_data_df['Country'] = wb_data_df['Country'].apply(lambda x: get_country_name(x))
wb_data_df.dropna(inplace=True)

In [44]:
wb_data_df.head()

Country  Year  Percent
660  Afghanistan  2017  4.05887
661  Afghanistan  2016  4.22836
662  Afghanistan  2015  3.25580
663  Afghanistan  2014  3.69522
664  Afghanistan  2013  3.45446

In [ ]:
# Save df to csv so we don't have to perform API calls again
#staging_df.to_csv(f"{staging_data_dir}/staging_df.csv")

#####            CSV Load

In [45]:
# Load raw CSV files to data frame
pisa_science = pd.read_csv(f"{source_data_dir}/pisa_science_2006_2015.csv")
pisa_math = pd.read_csv(f"{source_data_dir}/pisa_math_2003_2015.csv")
pisa_reading  = pd.read_csv(f"{source_data_dir}/pisa_read_2000_2015.csv")

### Data cleansing
----

In [46]:
# Options
# Delete nulls
# FillNA with zero, etc
# Reformat data (date format, name format)
# Save only the columns you need
#Get country name

#weather_df_corr = weather_df.corr()
#weather_df_corr.unstack().sort_values()

# Only pull data older than 2006
pisa_reading = pisa_reading.loc[pisa_reading['TIME']>=2006,['LOCATION','INDICATOR', 'TIME','Value'] ]
pisa_math = pisa_math.loc[pisa_math['TIME']>=2006,['LOCATION','INDICATOR', 'TIME','Value']]
pisa_science = pisa_science.loc[pisa_science['TIME']>=2006,['LOCATION','INDICATOR', 'TIME','Value']]

#Merge all Reading, Math, Science data
pisa_merged = pisa_reading.merge(pisa_math, how="inner", on=['LOCATION', 'TIME'], suffixes=('_READING', '_MATH'))
pisa_merged = pisa_merged.merge(pisa_science, how="inner", on=['LOCATION', 'TIME'], suffixes=('','_SCIENCE'))
pisa_merged = pisa_merged[['LOCATION','TIME', 'Value_READING','Value_MATH', 'Value']]
pisa_merged = pisa_merged.rename(columns={'LOCATION' : 'Country','TIME': 'Year', 'Value_READING':'Reading','Value_MATH':'Math', 'Value':'Science'})
pisa_merged['Average'] = pisa_merged[['Reading','Math','Science']].mean(axis=1).round(2)

# Rename ISO3 codes countries to their corresponding pycountry name
pisa_merged['Country'] = pisa_merged['Country'].apply(lambda x: get_country_name(x))
pisa_merged.dropna()
pisa_merged.head()

Country  Year  Reading  Math  Science  Average
0  Australia  2006      513   520      527   520.00
1  Australia  2009      515   514      527   518.67
2  Australia  2012      512   504      521   512.33
3  Australia  2015      503   494      510   502.33
4    Austria  2006      490   505      511   502.00

### Data analysis
----

#### Question 2: What is the education budget allocation effect on students' academic performance?
Data sources: OECD PISA Result<br/>
OECD Government budget allocation<br/>

In [50]:
# Perform statistical analysis
# Put applicable plot depending on your question

wb_pisa_merged_df = pisa_merged.merge(wb_data_df, how="inner", on=['Country', 'Year'])
wb_pisa_merged_df.drop(columns=['Reading', 'Math', 'Science'], axis=1, inplace=True)
#= pisa_merged.merge(pisa_science, how="inner", on=['LOCATION', 'TIME'], suffixes=('','_SCIENCE'))

In [51]:
wb_pisa_merged_df.

Country  Year  Average  Percent
0    Australia  2006   520.00  4.75359
1    Australia  2009   518.67  5.09327
2    Australia  2012   512.33  4.87765
3    Australia  2015   502.33  5.32175
4      Austria  2006   502.00  5.22652
..         ...   ...      ...      ...
119   Slovenia  2015   509.33  4.91156
120   Colombia  2015   410.33  4.47039
121  Hong Kong  2015   532.67  3.26212
122       Peru  2015   394.00  3.96680
123      Macao  2015   527.33  2.98170

[124 rows x 4 columns]